In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, pickle, time, gc, copy, sys
import warnings
import pydicom
import multiprocessing

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sys.path.append('../src')
from utils import ri, pickle_load, pickle_save
from preprocess_fold import data_split_StratifiedKFold

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import _LRScheduler

from sklearn import metrics
from utils_pytorch import cycle, CosineLR, AverageMeter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True

In [3]:
from albumentations import (
    Compose,
    RandomCrop,
    RandomBrightnessContrast,
    ShiftScaleRotate,
    Cutout,
)
from albumentations.core.transforms_interface import ImageOnlyTransform

# Config

In [4]:
# config
DEBUG = False # set False to do all process
output_dir = "../output"
preprocess_dir = "../input/preprocess"
df_train_path = "{}/df_train.csv".format(preprocess_dir)
train_npy_dir_path = "{}/train_law_npy".format(preprocess_dir)
os.makedirs(output_dir, exist_ok=True)

In [5]:
NUM_FOLD = 5
NUM_EPOCH = 8
NUM_CYCLE = 4
BATCH_SIZE = 64
FOLD_LIST = [1,2,3,4,5]
LR_RANGE = [1e-3, 1e-5]
STEP_PER_EPOCH = 256
MODEL_NAME = 'b0' # b0 or b2
if DEBUG:
    NUM_EPOCH = 1
    NUM_CYCLE = 1
    FOLD_LIST = [1]

In [6]:
col_index = 'SOPInstanceUID'
col_groupby = 'StudyInstanceUID'
col_targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
    'pe_present_on_image',
]
NUM_CLASS = len(col_targets)
print('NUM_CLASS: {}'.format(NUM_CLASS))

NUM_CLASS: 10


# Data Loading

In [7]:
# load train data
df_train = pd.read_csv(df_train_path)
print(df_train.shape)
df_train.head()

(1790594, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,6897fa9de148,2bfbb7fd2e8b,52b6b0b793bb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,1.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
2,6897fa9de148,2bfbb7fd2e8b,1997c99c9d59,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,2.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
3,6897fa9de148,2bfbb7fd2e8b,c6f29ac6659b,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,3.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
4,6897fa9de148,2bfbb7fd2e8b,487d9ab5531f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,4.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0


# Preprocessing

In [8]:
# make exam-level train data
df_train_exam = ri(df_train[df_train[col_groupby].duplicated()==False])
print(df_train_exam.shape)
df_train_exam.head()

(7279, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.338710,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,013358b540bb,2805267980e7,c0e16bbe96ef,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,145,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,1
2,0cee26703028,bac7becd2970,64bf37f1a302,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,144,17.0,0.118056,/mnt/disks/data6/rsna2020/preprocessed/train_l...,2
3,c28f3d01b14f,7d17c72fd0ce,265035a0f7aa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,105,6.0,0.057143,/mnt/disks/data6/rsna2020/preprocessed/train_l...,3
4,c8fbf1e08ac5,275497911f02,7a33bbba8386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,81,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,4


In [9]:
df_train_exam['start_index'] = df_train[df_train[col_groupby].duplicated()==False].index.values
df_train_exam.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index,start_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.338710,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0
1,013358b540bb,2805267980e7,c0e16bbe96ef,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,145,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,1,124
2,0cee26703028,bac7becd2970,64bf37f1a302,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,144,17.0,0.118056,/mnt/disks/data6/rsna2020/preprocessed/train_l...,2,269
3,c28f3d01b14f,7d17c72fd0ce,265035a0f7aa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,105,6.0,0.057143,/mnt/disks/data6/rsna2020/preprocessed/train_l...,3,413
4,c8fbf1e08ac5,275497911f02,7a33bbba8386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,81,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,4,518


# Data Splitting

In [10]:
# data splitting stratified by 3 classes, PE positive, PE negative or indeterminate
df_tmp = df_train_exam
df_tmp['3class'] = 1 # PE positive
df_tmp['3class'][df_tmp['negative_exam_for_pe']==1] = 0 # PE negative
df_tmp['3class'][df_tmp['indeterminate']==1] = 2 # indeterminate
df_fold_exam = data_split_StratifiedKFold(df_tmp, col_groupby, col_stratified='3class') # apply stratified K-fold
df_fold = pd.merge(df_train[[col_groupby]], df_fold_exam, on=col_groupby, how='left')
print(df_fold.shape)
print(df_fold_exam.shape)
df_fold_exam.head()

(1790594, 11)
(7279, 11)


,StudyInstanceUID,fold1_train,fold1_valid,fold2_train,fold2_valid,fold3_train,fold3_valid,fold4_train,fold4_valid,fold5_train,fold5_valid
0,6897fa9de148,1,0,0,1,1,0,1,0,1,0
1,013358b540bb,0,1,1,0,1,0,1,0,1,0
2,0cee26703028,1,0,1,0,0,1,1,0,1,0
3,c28f3d01b14f,1,0,1,0,1,0,1,0,0,1
4,c8fbf1e08ac5,0,1,1,0,1,0,1,0,1,0


# Definition of Model

In [11]:
class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Conv1d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv1d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x
    
class CNN_1D(nn.Module):

    def __init__(self, num_classes=400, input_ch=1, verbose=False):

        super(CNN_1D, self).__init__()
        pool = 4
        drop = 0.1
        self.verbose = verbose
        self.layer1 = nn.Sequential(
                nn.Conv1d(input_ch//pool, 64, kernel_size=7, stride=1, padding=3, bias=False),
                nn.BatchNorm1d(64),
                nn.ReLU(inplace=True),
                SEModule(64, 16),
        )
        self.fpool = nn.MaxPool1d(kernel_size=pool, stride=pool, padding=0)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer2 = nn.Sequential(
                nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(128),
                nn.ReLU(inplace=True),
                SEModule(128, 16),
        )
        self.layer3 = nn.Sequential(
                nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                SEModule(256, 16),
        )
        self.layer4 = nn.Sequential(
                nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                SEModule(512, 16),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc2 = nn.Conv1d(
            input_ch//pool+64+128+256+512, 
            2, kernel_size=1)
        self.fc = nn.Sequential(
                nn.Linear(512, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, 9),
        )

    def forward(self, x_input):
        bs, ch, d = x_input.size()
        x0 = torch.transpose(x_input, 1, 2)
        x0 = self.fpool(x0)
        x0 = torch.transpose(x0, 1, 2)
        x1 = self.layer1(x0)
        x1 = self.maxpool(x1)

        x2 = self.layer2(x1)
        x2 = self.maxpool(x2)
        x3 = self.layer3(x2)
        x3 = self.maxpool(x3)
        x4 = self.layer4(x3)
        y = self.avgpool(x4)
        y = y.view(bs, -1)
        y = self.fc(y)
        
        x5 = torch.cat([
            x0,
            F.adaptive_avg_pool1d(x1, d), 
            F.adaptive_avg_pool1d(x2, d), 
            F.adaptive_avg_pool1d(x3, d), 
            F.adaptive_avg_pool1d(x4, d), 
        ], axis=1)
        y2 = self.fc2(x5)
            
        return y, y2

# Definition of Training

In [12]:
class ImageDataset(Dataset):
    def __init__(self, X_exam, X_image, shuffle=False, crop=64, verbose=False, dropout=False, cropaug=0, flip=0, fold=1):
        self.X_exam = X_exam
        self.X_image = X_image
        self.shuffle = shuffle
        self.crop = crop
        self.verbose = verbose
        self.cropaug = cropaug # do extra crop augment or not
        self.flip = flip # do flip augmenet or not
        self.feature_dir = "{}/features_{}_fold{}".format(output_dir, MODEL_NAME, fold)

    def __getitem__(self, index):
        exam =  self.X_exam[col_groupby][index] # get the name of the selected exam
        num_series = self.X_exam['num_series'][index] # get the num of series in the selected exam
        
        # crop the data
        start_index = self.X_exam['start_index'][index]
        end_index = start_index + num_series
        if self.crop!=-1 and num_series>self.crop: # random crop in z-axis
            depth = self.crop
            tmp_index = np.random.randint(num_series-self.crop)
            df_tmp = ri(self.X_image.iloc[start_index+tmp_index:start_index+tmp_index+self.crop])
        else: # do not crop
            df_tmp = ri(self.X_image.iloc[start_index:end_index])
            depth = num_series
            tmp_index = 0
            
        # load features
        x_input = np.load("{}/{}_feature.npy".format(self.feature_dir, exam))[tmp_index:tmp_index+depth]
        x_input = x_input.astype(np.float32).transpose(1,0)

        # get targets
        # get the exam-level targets of the selected exam
        target1 = self.X_exam[col_targets[:-1]].values[index].astype(np.float32)
        # get the image-level targets of the selected exam
        target2 = df_tmp[col_targets[-1]].values.astype(np.float32).reshape([1, -1])
        # get the q_i weigh of the selected exam
        q_weight = self.X_exam['q_i'].values[index].astype(np.float32).reshape([1])
        
        # do padding if length of exam data is shoreter than the defined length
        if x_input.shape[1]<self.crop and self.crop!=-1:
            x_input_base = np.zeros([x_input.shape[0], self.crop], np.float32)
            x_input_base[:, :x_input.shape[1]] = x_input
            x_input = x_input_base
            target2_base = np.zeros([1, self.crop], np.float32)
            target2_base[:,:target2.shape[1]] = target2
            target2 = target2_base
            
        # do padding to make length multiple of 64
        if self.crop==-1:
            len_pad = int(np.ceil(x_input.shape[1]/64)*64)
            x_input_base = np.zeros([x_input.shape[0], len_pad], np.float32)
            x_input_base[:, :x_input.shape[1]] = x_input
            x_input = x_input_base
            target2_base = np.zeros([1, len_pad], np.float32)
            target2_base[:,:target2.shape[1]] = target2
            target2 = target2_base
            
        # do extra crop augment (crop both edges and do padding)
        if self.cropaug>0:
            l = np.random.randint(self.cropaug)
            r = np.random.randint(self.cropaug)
            x_input[:, :l] = 0
            x_input[:, -r:] = 0
            target2[:, :l] = 0
            target2[:, -r:] = 0

        # do flip augment
        if np.random.rand()<self.flip:
            x_input = np.copy(x_input[:,::-1])
            target2 = np.copy(target2[:,::-1])
            
        return x_input, (target1, target2, q_weight, num_series)
    
    def __len__(self):
        return len(self.X_exam)

In [13]:
# set exam-level class weights
class_weights = np.array([
    0.0736196319, 
    0.09202453988, 
    0.1042944785, 
    0.1042944785, 
    0.1877300613, 
    0.06257668712, 
    0.06257668712,
    0.2346625767,
    0.0782208589,
], np.float32)

class_weights = torch.from_numpy(class_weights).cuda()

In [14]:
# define training
def train(loader, model, optimizer, scheduler, num_step=-1, verbose=10):
    if num_step==-1: num_step = len(loader)
    loss1_avr = AverageMeter()
    loss2_avr = AverageMeter()
    loss3_avr = AverageMeter()
    
    criterion1 = nn.BCELoss().cuda()
    criterion2 = nn.BCELoss(reduce=False).cuda()
    lastfunc = nn.Sigmoid().cuda()

    # switch to train mode
    model .train()

    starttime = time.time()
    # training
    itr = cycle(loader)
    for i in range(num_step):
        if i+1>num_step: break
        # load batch data
        input1, (target1, target2, q_weight, _) = next(itr)
        input1 = torch.autograd.Variable(input1.cuda()) # input feature sequence
        target1 = torch.autograd.Variable(target1.cuda()) # exam-level targets
        target2 = torch.autograd.Variable(target2.cuda()) # image-level targets
        q_weight = torch.autograd.Variable(q_weight.cuda()) # q weight of image
        lower_limit = torch.from_numpy(np.ones([input1.size(0), 1], np.float32)*1e-3).cuda() # lower limit of q_i
        q_weight = torch.max(q_weight, lower_limit) # apply lower limit to q weight

        # compute output
        output1, output23 = model(input1)
        output2 = output23[:,:-1]
        output3 = output23[:,-1:]
        output1 = lastfunc(output1) # exam-level output [bs, 9]
        output2 = lastfunc(output2) # image-level output [bs, 1, depth]
        output3 = lastfunc(output3) # image-level output for q weighted loss [bs, 1, depth]

        # calc losses
        # exam-level weighted BCE
        loss1 = criterion2(output1, target1)
        loss1 = torch.mean(loss1, axis=0)
        loss1 = torch.sum(loss1 * class_weights)
        # image-level BCE
        loss2 = criterion1(output2, target2)
        # image-level q-weighted BCE
        loss3 = criterion2(output3, target2)
        loss3 = torch.mean(loss3, axis=(2))
        loss3 = torch.mean(loss3 * q_weight)/0.053915069524371764 # 0.053915069524371764 is the mean q weight of train data
        # total loss
        loss = loss1*0.45 + loss2 * 0.45 + loss3*0.1 
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # record losses
        loss1_avr.update(loss1.data, input1.size(0))
        loss2_avr.update(loss2.data, input1.size(0))
        loss3_avr.update(loss3.data, input1.size(0))
        
        # display log
        if (i+1)%verbose==0:
            print("Step: {}/{} ".format(i + 1, num_step)
                  + "Loss 1: {:.3f} ".format(loss1_avr.avg.item())
                  + "Loss 2: {:.3f} ".format(loss2_avr.avg.item())
                  + "Loss 3: {:.3f} ".format(loss3_avr.avg.item())
                  + "Sec: {:.1f} ".format(time.time()-starttime)
                  )
            
    return loss1_avr.avg.item() , loss2_avr.avg.item(), loss3_avr.avg.item()

In [15]:
def validate(loader, model, verbose=10):
    loss1_avr = AverageMeter()
    loss2_avr = AverageMeter()
    loss3_avr = AverageMeter()

    criterion1 = nn.BCELoss().cuda()
    criterion2 = nn.BCELoss(reduce=False).cuda() #.half()
    lastfunc = nn.Sigmoid()

    # switch to eval mode
    model .eval()

    starttime = time.time()
    # validation
    preds = np.zeros([0, NUM_CLASS-1], np.float32)
    preds2 = np.zeros([0, 1], np.float32)
    preds3 = np.zeros([0, 1], np.float32)
    y_true = np.zeros([0, NUM_CLASS-1], np.float32)
    y_true2 = np.zeros([0, 1], np.float32)
    for i, (input1, (target1, target2, q_weight, num_series)) in enumerate(loader):
        with torch.no_grad():
            # load batch data
            input1 = torch.autograd.Variable(input1.cuda())
            target1 = torch.autograd.Variable(target1.cuda())
            target2 = torch.autograd.Variable(target2.cuda())[:, :, :num_series]
            q_weight = torch.autograd.Variable(q_weight.cuda())

            output1, output23 = model(input1)
            output2 = output23[:,:-1]
            output3 = output23[:,-1:]
            output1 = lastfunc(output1)
            output2 = lastfunc(output2)[:, :, :num_series]
            output3 = lastfunc(output3)[:, :, :num_series]
            
            # calc losses
            loss1 = criterion2(output1, target1)
            loss1 = torch.mean(loss1, axis=0)
            loss1 = torch.sum(loss1 * class_weights)
            # image-level BCE
            loss2 = criterion1(output2, target2)
            # image-level q-weighted BCE
            loss3 = criterion2(output3, target2)
            loss3 = torch.mean(loss3, axis=(2))
            loss3 = torch.mean(loss3 * q_weight)/0.053915069524371764 
            
        # record losses
        loss1_avr.update(loss1.data, input1.size(0))
        loss2_avr.update(loss2.data, input1.size(0))
        loss3_avr.update(loss3.data, input1.size(0))

        # record pred and true data
        # exam-level pred
        preds = np.concatenate([preds, output1.data.cpu().numpy()]) 
        # image-level pred
        preds2 = np.concatenate([preds2, output2.data.cpu().numpy().reshape([-1, 1])])
        # image-level pred optimized to q-weighted BCE
        preds3 = np.concatenate([preds3, output3.data.cpu().numpy().reshape([-1, 1])])
        # exam-level true data
        y_true = np.concatenate([y_true, target1.data.cpu().numpy()])
        # image-level true data
        y_true2 = np.concatenate([y_true2, target2.data.cpu().numpy().reshape([-1, 1])])
        
        # display log
        if (i+1)%verbose==0:
            print("Step: {}/{} ".format(i + 1, len(loader))
                  + "Loss 1: {:.3f} ".format(loss1_avr.avg.item())
                  + "Loss 2: {:.3f} ".format(loss2_avr.avg.item())
                  + "Loss 3: {:.3f} ".format(loss3_avr.avg.item())
                  + "Sec: {:.1f} ".format(time.time()-starttime)
                  )
        
    return loss1_avr.avg.item(), loss2_avr.avg.item(), loss3_avr.avg.item(), preds, preds2, preds3, y_true, y_true2

# Training

In [16]:
# set train log
log_columns = ['epoch', 'loss1', 'loss2', 'loss3', 'val_loss1', 'val_loss2', 'val_loss3', 'time']
for col in col_targets:
    log_columns.append("val_bce_{}".format(col))
    log_columns.append("val_auc_{}".format(col))
log_columns.append("val_bce_q_{}".format(col))
log_columns.append("val_auc_q_{}".format(col))
log_columns.append("val_loss")

In [17]:
# training
for fold in range(NUM_FOLD):
    if fold+1 not in FOLD_LIST: continue
    starttime = time.time()
    print("fold: {}".format(fold + 1))

    # build model
    if MODEL_NAME=='b0':
        num_feature = 1280
    elif MODEL_NAME=='b2':
        num_feature = 1408
    model = CNN_1D(num_classes=NUM_CLASS+1, input_ch=num_feature).cuda()
    
    # train dataset
    X_train_exam = ri(df_train_exam[df_fold_exam['fold{}_train'.format(fold+1)]==1])
    dataset_train = ImageDataset(X_train_exam, df_train, shuffle=True, 
                                 crop=128,
                                 cropaug=32,
                                 flip=0.5,
                                 fold=fold+1,
                                )
    train_loader = DataLoader(dataset_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              worker_init_fn=lambda x: np.random.seed(),
                              )

    # valid dataset
    X_valid_exam = ri(df_train_exam[df_fold_exam['fold{}_valid'.format(fold+1)]==1])
    dataset_valid = ImageDataset(X_valid_exam, df_train, shuffle=False, crop=-1,fold=fold+1)
    valid_loader = DataLoader(dataset_valid,
                              batch_size=1,
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              )
    
    # set optimizer and loss
    optimizer = optim.Adam(model.parameters(), lr=LR_RANGE[0])
    scheduler = CosineLR(optimizer, step_size_min=LR_RANGE[1], t0=STEP_PER_EPOCH * NUM_CYCLE, tmult=1)
    
    # set train log
    train_log = pd.DataFrame(columns=log_columns)
    
    # training
    for epoch in range(NUM_EPOCH):
        loss1, loss2, loss3 = train(train_loader, model, optimizer, scheduler, num_step=STEP_PER_EPOCH, verbose=100)
        val_loss1, val_loss2, val_loss3, val_pred, val_pred2, val_pred3, val_true, val_true2 = validate(valid_loader, model, verbose=100)

        # record log
        endtime = time.time() - starttime
        print_log = "Epoch: {}/{} ".format(epoch + 1, NUM_EPOCH)
        print_log += "Sec: {:.1f} \n".format(time.time()-starttime)
        print_log += "BCE 1: {:.3f} ".format(loss1)
        print_log += "BCE 2: {:.3f} ".format(loss2)
        print_log += "BCE 3: {:.3f} \n".format(loss3)
        print_log += "val BCE 1: {:.3f} ".format(val_loss1)
        print_log += "val BCE 2: {:.3f} ".format(val_loss2)
        print_log += "val BCE 3: {:.3f} \n".format(val_loss3)
        train_log_epoch = [epoch+1, loss1, loss2, loss3, val_loss1, val_loss2, val_loss3, endtime]
        
        for i, col in enumerate(col_targets[:-1]):
            score = metrics.log_loss(val_true[:,i], val_pred[:,i], labels=[0,1])
            train_log_epoch.append(score)
            print_log += "val bce {}: {:.3f} ".format(col, score)
            
            score = metrics.roc_auc_score(val_true[:,i], val_pred[:,i])
            train_log_epoch.append(score)
            print_log += "val auc {}: {:.3f} \n".format(col, score)
        
        score = metrics.log_loss(val_true2[:,0], val_pred2[:,0], labels=[0,1])
        train_log_epoch.append(score)
        print_log += "val bce {}: {:.3f} ".format(col_targets[-1], score)

        score = metrics.roc_auc_score(val_true2[:,0], val_pred2[:,0])
        train_log_epoch.append(score)
        print_log += "val auc {}: {:.3f} \n".format(col_targets[-1], score)
        
        pred_tmp = np.clip(val_pred3[:,0], 1e-15, 1 - 1e-15)
        true_tmp = df_train['pe_present_on_image'].values[df_fold['fold{}_valid'.format(fold+1)]==1]
        q_tmp = df_train['q_i'].values[df_fold['fold{}_valid'.format(fold+1)]==1]
        tmp = -q_tmp*(np.log(pred_tmp)*true_tmp + np.log(1-pred_tmp)*(1-true_tmp))
        score = np.sum(tmp)/np.sum(q_tmp)
        train_log_epoch.append(score)
        print_log += "val bce q_{}: {:.3f} ".format(col_targets[-1], score)
        val_loss = (val_loss1 + score)/2

        score = metrics.roc_auc_score(val_true2[:,0], val_pred3[:,0])
        train_log_epoch.append(score)
        print_log += "val auc q_{}: {:.3f} \n".format(col_targets[-1], score)
        
        train_log_epoch.append(val_loss)
        print_log += "val loss : {:.3f} \n".format(val_loss)
        
        train_log_epoch = pd.DataFrame([train_log_epoch], columns=log_columns)
        train_log = pd.concat([train_log, train_log_epoch])
        train_log.to_csv("{}/1dcnn_train_log_{}_fold{}.csv".format(output_dir, MODEL_NAME, fold + 1), index=False)

        # display log
        print(print_log)
        
        # save weights
        if val_loss==train_log['val_loss'].min():
            torch.save(model.state_dict(), "{}/1dcnn_weight_{}_best_fold{}.pth".format(
                output_dir, MODEL_NAME, fold + 1))
        if (epoch+1)%NUM_CYCLE==0:
            torch.save(model.state_dict(), "{}/1dcnn_weight_{}_epoch_{}_fold{}.pth".format(
                    output_dir, MODEL_NAME, epoch+1, fold + 1))
            torch.save(optimizer.state_dict(), "{}/1dcnn_optimizer_{}_epoch_{}_fold{}.pth".format(
                    output_dir, MODEL_NAME, epoch+1, fold + 1))

fold: 1
Step: 100/256 Loss 1: 0.258 Loss 2: 0.110 Loss 3: 0.266 Sec: 103.7 
Step: 200/256 Loss 1: 0.236 Loss 2: 0.096 Loss 3: 0.243 Sec: 173.1 
Step: 100/1456 Loss 1: 0.237 Loss 2: 0.101 Loss 3: 0.391 Sec: 3.2 
Step: 200/1456 Loss 1: 0.217 Loss 2: 0.103 Loss 3: 0.386 Sec: 4.8 
Step: 300/1456 Loss 1: 0.219 Loss 2: 0.096 Loss 3: 0.320 Sec: 6.5 
Step: 400/1456 Loss 1: 0.216 Loss 2: 0.095 Loss 3: 0.291 Sec: 8.1 
Step: 500/1456 Loss 1: 0.211 Loss 2: 0.090 Loss 3: 0.273 Sec: 9.8 
Step: 600/1456 Loss 1: 0.205 Loss 2: 0.087 Loss 3: 0.266 Sec: 11.2 
Step: 700/1456 Loss 1: 0.200 Loss 2: 0.081 Loss 3: 0.244 Sec: 12.8 
Step: 800/1456 Loss 1: 0.202 Loss 2: 0.083 Loss 3: 0.264 Sec: 14.1 
Step: 900/1456 Loss 1: 0.199 Loss 2: 0.082 Loss 3: 0.259 Sec: 15.6 
Step: 1000/1456 Loss 1: 0.203 Loss 2: 0.084 Loss 3: 0.261 Sec: 17.2 
Step: 1100/1456 Loss 1: 0.204 Loss 2: 0.083 Loss 3: 0.262 Sec: 18.8 
Step: 1200/1456 Loss 1: 0.208 Loss 2: 0.083 Loss 3: 0.257 Sec: 20.8 
Step: 1300/1456 Loss 1: 0.210 Loss 2: 0.08

# Prediction

In [18]:
def predict(loader, model, verbose=100):
    lastfunc = nn.Sigmoid()

    # switch to eval mode
    model .eval()

    starttime = time.time()
    # training
    preds = np.zeros([0, NUM_CLASS-1], np.float32)
    preds2 = np.zeros([0, 1], np.float32)
    preds3 = np.zeros([0, 1], np.float32)
    for i, (input1, (_, _, _, num_series)) in enumerate(loader):
        with torch.no_grad():
            input1 = torch.autograd.Variable(input1.cuda())
            # compute output
            output1, output23 = model(input1)
            output2 = output23[:,:-1]
            output3 = output23[:,-1:]
            output1 = lastfunc(output1)
            output2 = lastfunc(output2)[:,:,:num_series]
            output3 = lastfunc(output3)[:,:,:num_series]
        
        # record pred
        preds = np.concatenate([preds, output1.data.cpu().numpy()])
        preds2 = np.concatenate([preds2, output2.data.cpu().numpy().reshape([-1, 1])])
        preds3 = np.concatenate([preds3, output3.data.cpu().numpy().reshape([-1, 1])])
        
        # display log
        if (i+1)%verbose==0:
            print("Step: {}/{} ".format(i + 1, len(loader))
                  + "Sec: {:.1f} ".format(time.time()-starttime)
                  )
        
    return preds, preds2, preds3

In [19]:
# bulid model
if MODEL_NAME=='b0':
    num_feature = 1280
elif MODEL_NAME=='b2':
    num_feature = 1408
model = CNN_1D(num_classes=NUM_CLASS+1, input_ch=num_feature).cuda()

In [20]:
for fold in range(NUM_FOLD):
    if fold+1 not in FOLD_LIST: continue
    # valid dataset
    X_valid_study = ri(df_train_exam[df_fold_exam['fold{}_valid'.format(fold+1)]==1])
    dataset_valid = ImageDataset(X_valid_study, df_train, shuffle=False, crop=-1, fold=fold+1)
    valid_loader = DataLoader(dataset_valid,
                              batch_size=1,
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count(),
                              pin_memory=True,
                              )
    
    model.load_state_dict(torch.load("{}/1dcnn_weight_{}_best_fold{}.pth".format(
                    output_dir, MODEL_NAME, epoch+1, fold + 1)))
    preds_valid1, preds_valid2, preds_valid3 = predict(valid_loader, model, verbose=100)
    np.save("{}/1dcnn_preds_{}_valid1_fold{}.npy".format(output_dir, MODEL_NAME, fold+1), preds_valid1)
    np.save("{}/1dcnn_preds_{}_valid2_fold{}.npy".format(output_dir, MODEL_NAME, fold+1), preds_valid2)
    np.save("{}/1dcnn_preds_{}_valid3_fold{}.npy".format(output_dir, MODEL_NAME, fold+1), preds_valid3)

Step: 100/1456 Sec: 0.9 
Step: 200/1456 Sec: 1.5 
Step: 300/1456 Sec: 2.1 
Step: 400/1456 Sec: 2.8 
Step: 500/1456 Sec: 3.4 
Step: 600/1456 Sec: 4.1 
Step: 700/1456 Sec: 4.7 
Step: 800/1456 Sec: 5.4 
Step: 900/1456 Sec: 6.0 
Step: 1000/1456 Sec: 6.6 
Step: 1100/1456 Sec: 7.3 
Step: 1200/1456 Sec: 7.9 
Step: 1300/1456 Sec: 8.6 
Step: 1400/1456 Sec: 9.3 
